<a href="https://colab.research.google.com/github/dmmmit/booking_canceller/blob/main/stacking_cat_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import torch
import tensorflow as tf


DEFAULT_RANDOM_SEED = 42


def set_all_seeds(seed=DEFAULT_RANDOM_SEED):

    # python's seeds
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    # torch's seeds
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # tensorflow's seed
    tf.random.set_seed(seed)


set_all_seeds(seed=DEFAULT_RANDOM_SEED)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install openpyxl catboost >> None

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

In [ ]:
train_data = pd.read_excel('train.xlsx')
test_data = pd.read_excel('test.xlsx')

if 'Статус брони' in train_data.columns:
    train_data = train_data.drop(columns=['Статус брони', 'Unnamed: 0'])
if 'Unnamed: 0' in test_data.columns:
    test_data = test_data.drop(columns=['Unnamed: 0'])


In [ ]:
if "№ брони" in train_data.columns:
    train_data.drop(columns=['№ брони'], inplace=True)
if "№ брони" in test_data.columns:
    test_data.drop(columns=['№ брони'], inplace=True)
train_data.sample(5)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
5405,1,13900.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-17 11:23:03,NaT,2022-05-28 15:00:00,1,2022-05-29 12:00:00,Официальный сайт,Номер «Стандарт»,2,2
20995,1,8700.0,0,При заселении,2023-04-19 09:32:46,NaT,2023-04-22 15:00:00,1,2023-04-23 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
15439,1,28800.0,28800,Банк. карта: Банк Россия (банк. карта),2023-08-04 00:47:36,NaT,2023-08-17 15:00:00,2,2023-08-19 12:00:00,Программа лояльности,Номер «Стандарт»,2,3
19500,1,23200.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-10 19:41:15,NaT,2023-08-15 15:00:00,1,2023-08-16 12:00:00,Официальный сайт,Номер «Студия»,2,4
17112,1,13900.0,13900,Отложенная электронная оплата: Банк Россия (ба...,2022-08-22 11:12:58,NaT,2022-08-29 15:00:00,1,2022-08-30 12:00:00,Бронирование из экстранета,Номер «Стандарт»,2,2


In [ ]:
train_date_columns = ['Дата бронирования', 'Дата отмены', 'Заезд', 'Выезд']
test_date_columns = ['Дата бронирования', 'Заезд', 'Выезд']

for col in train_date_columns:
    train_data[col] = pd.to_datetime(train_data[col], errors='coerce')

for col in test_date_columns:
    test_data[col] = pd.to_datetime(test_data[col], errors='coerce')


In [ ]:
def extract_date_features(df, is_train=True):
    df['День бронирования'] = df['Дата бронирования'].dt.day
    df['Месяц бронирования'] = df['Дата бронирования'].dt.month
    df['День недели бронирования'] = df['Дата бронирования'].dt.weekday
    df['Количество номеров'] = df['Категория номера'].apply(lambda x: len(x.split('\n')) if pd.notnull(x) else 0)
    df['День заезда'] = df['Заезд'].dt.day
    df['Месяц заезда'] = df['Заезд'].dt.month
    df['День недели заезда'] = df['Заезд'].dt.weekday

    df['Количество дней'] = (df['Заезд'] - df['Дата бронирования']).dt.days
    df['prepayment_ratio'] = df['Внесена предоплата'] / df['Стоимость']
    df['guests_per_room'] = df['Гостей'] / df['Номеров']


    if is_train:
        # Дополнительные признаки для тренировочного набора, если необходимо
        pass

    return df


train_data = extract_date_features(train_data, is_train=True)
test_data = extract_date_features(test_data, is_train=False)

In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Зима'
    elif month in [3, 4, 5]:
        return 'Весна'
    elif month in [6, 7, 8]:
        return 'Лето'
    else:
        return 'Осень'

test_data['booking_season'] = test_data['Месяц бронирования'].apply(get_season)
train_data['booking_season'] = train_data['Месяц бронирования'].apply(get_season)

In [ ]:
# Список категориальных признаков
categorical_features = ['Способ оплаты', 'Источник', 'Категория номера', 'Гостиница', 'booking_season']

# Проверяем наличие пропущенных значений в категориальных признаках тестового набора
print("Пропущенные значения в категориальных признаках тестового набора:")
print(test_data[categorical_features].isnull().sum())


Пропущенные значения в категориальных признаках тестового набора:
Способ оплаты       0
Источник            0
Категория номера    0
Гостиница           0
booking_season      0
dtype: int64


In [ ]:
for col in categorical_features:
    train_data[col] = train_data[col].astype(str).fillna('Missing')
    test_data[col] = test_data[col].astype(str).fillna('Missing')


In [ ]:
train_data['Отмена'] = train_data['Дата отмены'].notnull().astype(int)

In [ ]:
numerical_features = ['Номеров', 'Стоимость', 'Внесена предоплата', 'Ночей', 'Гостей',
                      'День бронирования', 'Месяц бронирования', 'День недели бронирования',
                      'День заезда', 'Месяц заезда', 'День недели заезда',
                      'Количество дней', 'prepayment_ratio','guests_per_room','Количество номеров']


for col in numerical_features:
    train_data[col] = pd.to_numeric(train_data[col], errors='coerce')
    test_data[col] = pd.to_numeric(test_data[col], errors='coerce')

    # Заполняем пропущенные значения медианой
    median = train_data[col].median()
    train_data[col].fillna(median, inplace=True)
    test_data[col].fillna(median, inplace=True)

In [ ]:
features = numerical_features + categorical_features


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26174 entries, 0 to 26173
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Номеров                   26174 non-null  int64         
 1   Стоимость                 26174 non-null  float64       
 2   Внесена предоплата        26174 non-null  int64         
 3   Способ оплаты             26174 non-null  object        
 4   Дата бронирования         26174 non-null  datetime64[ns]
 5   Дата отмены               5192 non-null   datetime64[ns]
 6   Заезд                     26174 non-null  datetime64[ns]
 7   Ночей                     26174 non-null  int64         
 8   Выезд                     26174 non-null  datetime64[ns]
 9   Источник                  26174 non-null  object        
 10  Категория номера          26174 non-null  object        
 11  Гостей                    26174 non-null  int64         
 12  Гостиница         

In [ ]:
xgb_model = xgb.XGBClassifier(
    eta=0.05,
    n_estimators=1500,
    max_depth=6,
    subsample=0.7,
    min_child_weight=0.1,
    gamma=0.01,
    reg_lambda=0.1,
    reg_alpha=0.5,
    objective="binary:logistic",
    eval_metric="auc",

    gpu_id=0,
    enable_categorical=True
)

cat_model = cb.CatBoostClassifier(

    n_estimators=1000,
    learning_rate=0.03,
    depth=5,
    verbose=False,
    l2_leaf_reg=3,
    bagging_temperature=0.5,
    rsm=0.7,
    loss_function='Logloss',
    auto_class_weights='Balanced',
    random_state=42
)

In [ ]:
!pip3 install -U scikit-learn==1.2.2

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

In [ ]:
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])


In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)
])


In [ ]:
X = train_data[features]
y = train_data['Отмена']
X_test = test_data[features]

In [ ]:
stacking_model = StackingClassifier(
    estimators=[
        ('xgb', xgb_model),
        ('cat', cat_model)
    ],
    final_estimator=LogisticRegression(),
    stack_method='predict_proba'
)

model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("stacking_model", stacking_model)
])

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f"Фолд {fold+1}")

    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model_pipeline.fit(X_train, y_train)

    y_pred_proba_val = model_pipeline.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba_val)
    auc_scores.append(auc)

    print(f"ROC-AUC на валидационном наборе: {auc:.4f}")

print(f"Средний ROC-AUC по всем фолдам: {np.mean(auc_scores):.4f}")
y_pred_proba_test = model_pipeline.predict_proba(X_test)[:, 1]

pd.DataFrame(y_pred_proba_test, columns=['probability']).to_csv('predictions_stack.csv', index=False)


Фолд 1
ROC-AUC на валидационном наборе: 0.8550
Фолд 2
ROC-AUC на валидационном наборе: 0.8545
Фолд 3
ROC-AUC на валидационном наборе: 0.8453
Фолд 4
ROC-AUC на валидационном наборе: 0.8642
Фолд 5
ROC-AUC на валидационном наборе: 0.8612
Средний ROC-AUC по всем фолдам: 0.8560
